In [1]:
import gc
import os
import sys
import math
import random
import warnings
import pickle
from pathlib import Path
from joblib import Parallel, delayed
from statsmodels.tsa.deterministic import (CalendarFourier,
                                           CalendarSeasonality,
                                           CalendarTimeTrend,
                                           DeterministicProcess)

from tqdm import tqdm
import os
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
# from pandarallel import pandarallel
# pandarallel.initialize()
warnings.simplefilter("ignore")
import ctypes as ct
from datetime import timedelta

from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import PowerTransformer

import lightgbm as lgbm


In [2]:
sys.path.append('../../')
import src.utils as utils

In [3]:
EXP_NUM = 1

In [4]:
DATA_DIR = Path("/home/knikaido/work/MLB-Player-Digital-Engagement-Forecasting/data/")
MAIN_DATA_DIR = DATA_DIR / 'mlb-player-digital-engagement-forecasting'
TRAIN_DIR = MAIN_DATA_DIR / 'train'
OUTPUT_DIR = Path('./output/')

In [5]:
players = pd.read_csv(MAIN_DATA_DIR / 'players.csv')

rosters = pd.read_csv(TRAIN_DIR / 'rosters_train.csv')
targets = pd.read_csv(TRAIN_DIR / 'nextDayPlayerEngagement_train.csv')
scores = pd.read_csv(TRAIN_DIR / 'playerBoxScores_train.csv')
scores = scores.groupby(['playerId', 'date']).sum().reset_index()
seasons = pd.read_csv(MAIN_DATA_DIR / 'seasons.csv')
twitters = pd.read_csv(TRAIN_DIR / 'playerTwitterFollowers_train.csv')
transactions = pd.read_csv(TRAIN_DIR / 'transactions_train.csv')
awards = pd.read_csv(TRAIN_DIR / 'awards_train.csv')


In [9]:
targets = pd.read_csv(TRAIN_DIR / 'nextDayPlayerEngagement_train.csv')
targets = pd.merge(targets, players, how='left', on='playerId')

In [12]:
targets[targets['playerName']=='Shohei Ohtani']['target2'].sort_values()

2095609      0.249173
2097153      0.252476
2093286      0.288136
2103698      0.297249
2101223      0.337856
              ...    
214749     100.000000
217771     100.000000
2420562    100.000000
200191     100.000000
265889     100.000000
Name: target2, Length: 1216, dtype: float64

In [6]:
no_roster = np.array(list(set((players['playerId'].unique())) - set((rosters['playerId'].unique())) ))

In [8]:
players[players['playerName']=='Shohei Ohtani']

,playerId,playerName,DOB,mlbDebutDate,birthCity,birthStateProvince,birthCountry,heightInches,weight,primaryPositionCode,primaryPositionName,playerForTestSetAndFuturePreds
798,660271,Shohei Ohtani,1994-07-05,2018-03-29,Oshu,NaN,Japan,76,210,1,Pitcher,True


In [103]:
players[players['playerId'].isin(no_roster)]

,playerId,playerName,DOB,mlbDebutDate,birthCity,birthStateProvince,birthCountry,heightInches,weight,primaryPositionCode,primaryPositionName,playerForTestSetAndFuturePreds
1,593590,Webster Rivas,1990-08-08,2021-05-28,Nagua,NaN,Dominican Republic,73,219,3,First Base,True
4,666201,Alek Manoah,1998-01-09,2021-05-27,Homestead,FL,USA,78,260,1,Pitcher,True
5,680911,Owen Miller,1996-11-15,2021-05-23,Mequon,WI,USA,72,185,4,Second Base,True
7,608422,Jose Godoy,1994-10-13,2021-05-21,Maracaibo,NaN,Venezuela,71,200,2,Catcher,True
10,664731,P.J. Higgins,1993-05-10,2021-05-19,Bridgeport,CT,USA,70,195,2,Catcher,True
12,641561,Johneshwy Fargas,1994-12-15,2021-05-17,Trujillo Alto,NaN,Puerto Rico,73,180,8,Outfielder,True
14,667297,Tommy Nance,1991-03-19,2021-05-17,Long Beach,CA,USA,78,235,1,Pitcher,True
15,622668,Ivan Castillo,1995-05-30,2021-05-14,Santiago,NaN,Dominican Republic,69,179,4,Second Base,True
16,669302,Logan Gilbert,1997-05-05,2021-05-13,Winter Park,FL,USA,78,225,1,Pitcher,True
17,672284,Jarred Kelenic,1999-07-16,2021-05-13,Waukesha,WI,USA,73,190,7,Outfielder,True
